In [21]:
import pandas as pd
import datetime as date
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import covidcast
from bs4 import BeautifulSoup
import requests
from sodapy import Socrata
import covidcast
import warnings
warnings.filterwarnings('ignore')

Notes:

Data Sites:

covid19-projections.com
worldmeters.info
covid tracking project
individual sites
CDC flu tracker
individual state sites
USC behavior data
HHS data
CDC case level data
Our World in Data
mortality.org
IHME data


### Covid 19 Projections

In [2]:
c19_url = "https://raw.githubusercontent.com/youyanggu/covid19_projections/master/infection_estimates/latest_all_estimates_states.csv"
c19_url_usa = "https://raw.githubusercontent.com/youyanggu/covid19_projections/master/infection_estimates/latest_all_estimates_us.csv"
#hhs_url = 'https://healthdata.gov/sites/default/files/reported_hospital_capacity_admissions_facility-level_weekly_average_timeseries_20201207.csv'

In [4]:
c19_df_states = pd.read_csv(c19_url)
c19_df_usa = pd.read_csv(c19_url_usa)

In [5]:
c19_df_states = c19_df_states.fillna(0)
c19_df_states['date'] = pd.to_datetime(c19_df_states['date'], format="%Y-%m-%d")
c19_df_usa = c19_df_usa.fillna(0)
c19_df_usa['date'] = pd.to_datetime(c19_df_usa['date'], format="%Y-%m-%d")

In [6]:
# example for daily printout readings

date = c19_df_usa['date'].iloc[-1]
cases = c19_df_usa['daily_positive_7day_ma'].iloc[-1]
tests = c19_df_usa['daily_tests_7day_ma'].iloc[-1]
adj_pos_rate = c19_df_usa['positivity_rate_7day_ma'].iloc[-1]

print(f'The report today is from {date:%Y-%m-%d} and the 7d MA is {cases:,} on {tests:,} tests \
with an adjusted positivity rate of {adj_pos_rate:.2%} data: covid19-projections.com')

The report today is from 2021-02-19 and the 7d MA is 68,933.4 on 1,457,820.0 tests with an adjusted positivity rate of 4.73% data: covid19-projections.com


### Worldmeters

In [3]:
url = 'https://www.worldometers.info/coronavirus/country/us/'

header = {
  "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
  "X-Requested-With": "XMLHttpRequest"
}

r = requests.get(url, headers=header)

wom_df = pd.read_html(r.text)
    


In [4]:
wow_df_today = pd.DataFrame(wom_df[0])

In [6]:
wow_df_yesterday = pd.DataFrame(wom_df[1])

### Oxford SI

In [7]:
oxford = 'https://raw.githubusercontent.com/OxCGRT/USA-covid-policy/master/data/OxCGRT_US_latest.csv'

In [8]:
oxford_data = pd.read_csv(oxford)

### HHS Datasets

In [11]:
client = Socrata("beta.healthdata.gov", None)

In [12]:
results = client.get("g62h-syeh", limit=16000)

In [13]:
hhs_ts_df = pd.DataFrame.from_records(results)

In [14]:
hhs_ts_df['date'] = pd.to_datetime(hhs_ts_df['date'])

In [15]:
for col in hhs_ts_df.columns[2:60]:
    hhs_ts_df[col] = pd.to_numeric(hhs_ts_df[col], errors='coerce')

#### daily HHS

In [17]:
results_daily = client.get("6xf2-c3ie", limit=2000)

# Convert to pandas DataFrame
hhs_daily_df = pd.DataFrame.from_records(results_daily)

### Our World in Data 

In [20]:
owid_df = pd.read_csv('https://covid.ourworldindata.org/data/owid-covid-data.csv')

### Mask Compliance data from CMU

##### docs: https://cmu-delphi.github.io/covidcast/covidcast-py/html/

In [23]:
from datetime import date
cmu_county_df = covidcast.signal("fb-survey", "smoothed_wearing_mask",
                        date(2020, 10, 15), date(2020, 11, 29),
                        "county")

In [27]:
cmu_state_df = covidcast.signal("fb-survey", "smoothed_wearing_mask",
                        date(2020, 10, 1), date(2021, 2, 22),
                        "state")

/Users/mikecave/anaconda3/envs/working/lib/python3.8/site-packages/covidcast/covidcast.py:372: UserWarning: Problem obtaining data on 20210222: no results
  warnings.warn("Problem obtaining data on {day}: {message}".format(


### Old CTP Data

In [32]:
ctp_url = 'https://covidtracking.com/api/v1/states/daily.csv'
ctp_url_usa = 'https://covidtracking.com/api/v1/us/daily.csv'

In [33]:
ctp_df = pd.read_csv(ctp_url)
ctp_df = ctp_df.fillna(0)
ctp_df['date'] = pd.to_datetime(ctp_df['date'], format="%Y%m%d")

In [34]:
ctp_usa_df = pd.read_csv(ctp_url_usa)
ctp_usa_df = ctp_usa_df.fillna(0)
ctp_usa_df['date'] = pd.to_datetime(ctp_usa_df['date'], format="%Y%m%d")

In [35]:
# converts date column to more consumable option for differing types of visualizations
ctp_df['date'] = ctp_df['date'].dt.date
ctp_usa_df['date'] = ctp_usa_df['date'].dt.date